# アンドロイドセンサーデータをオブジェクトストレージに保存する

## 概要

AndroidからKafkaブローカに送信されたセンサーデータを外部のオブジェクトストレージに保存するように設定します。この設定はKafkaブローカに送信されたデータのバックアップなどを目的としています。

### システム構成

システム構成を次図に示します。

![システム構成](img/101-components.png)

<!--
```mermaid
flowchart LR
  S3[("Object Storage")]
  subgraph S1[server]
    subgraph N["NATS"]
      T0(["sensor data topic"])
    end
    subgraph K["Apache Kafka"]
      T1(["sensor data topic"])
    end
    T0-.->T1
  end
  T1-.->|"data"|S3
  T1-.->|"timestamp"|S3
  subgraph R["Android"]
    CC["sinetstream-android-\nsensor-publisher"]
    CC-.->T0
  end```
-->

### 前提条件

設定を行うための前提条件を示します。

* [01-setup-android-sensor-data.ipynb](../../../Server/01-setup-android-sensor-data.ipynb)でサーバが構築済みであること
* 保存先となるオブジェクトストレージにサーバ構築環境からアクセスできること

## パラメータの指定

設定を行うために必要となるパラメータを指定します。

### 配置場所 

サーバを構成する資材が配置されているディレクトリを次のセルに指定してください。

In [ ]:
# (例)
# target_dir=$HOME/srv/sensor-viewer
# target_dir=/srv/sensor-viewer

target_dir=

指定されたディレクトリに想定するサーバが構築されていることを確認します。構成ファイルが配置されており、かつandroidセンサーデータ向けにサーバが構築されていることを確認します。

In [ ]:
test -f ${target_dir}/params/00-config.yml && \
grep -q android ${target_dir}/params/00-config.yml

コンテナが実行中であることを確認します。

In [ ]:
docker compose --project-directory ${target_dir} ps --services --status running | \
grep -q kafka-connect

Kafka ConnectのREST APIにアクセスできることを確認します。

In [ ]:
curl -sf http://localhost:8083

### オブジェクトストレージ

データ保存先となるオブジェクトストレージに関するパラメータを指定します。

オブジェクトストレージのURLを指定して下さい。

In [ ]:
# (例)
# endpoint_url=http://bucket.s3-aws-region.amazonaws.com
# endpoint_url=http://192.168.10.100:9000

endpoint_url=

保存先となるバケットの名前を指定して下さい。保存先となるバケットは既に作成されていることを想定しています。

In [ ]:
# (例)
# bucket_name=android

bucket_name=

オブジェクトストレージにアクセスするためのアクセスキーを指定して下さい。

In [ ]:
# (例)
# access_key=access-key

access_key=

オブジェクトストレージにアクセスするためのシークレットキーを指定して下さい。

In [ ]:
# (例)
# secret_key=secret-key-change-me

secret_key=

### Kafka Connect

Kafkaブローカからオブジェクトストレージにメッセージを保存するKafka Connectに関するパラメータの指定を行います。

#### 登録名
Kafka Connectに登録する名前を指定します。

現在登録されているKafka Connectの名前を確認します。`jq`コマンドが利用できない場合は、次のセルの末尾の`| jq .`の部分をコメントアウトして実行して下さい。

In [ ]:
curl -sf http://localhost:8083/connectors | jq .

Kafka Connectの登録名を指定します。

In [ ]:
# (例)
# kafka_connect_name=sink-s3-android

kafka_connect_name=

#### トピック名

Androidセンサーデータの送信先となっているKafkaのトピック名を確認します。ここで確認したものがKafka Connectを登録する対象のトピック名になります。

In [ ]:
topic_name=$(grep sensor_data $target_dir/params/01-android.yml | \
    awk '{print $2}' | tr -d '"')
echo $topic_name

## Kafka Connectの設定

ブローカに送信されたセンサーデータをオブジェクトストレージに保存するKafka Connectを登録します。

### センサーデータ

センサーデータをオブジェクトストレージに保存するKafka Connectを登録します。

In [ ]:
script_dir=../../../Server/files/setup/kafka-connect
$script_dir/setup-sink-minio.sh -D -t $topic_name -b $bucket_name \
    -u $endpoint_url -A "$access_key" -S "$secret_key" \
    -n $kafka_connect_name -X .json

登録状況を確認します。`jq`コマンドが利用できない場合は、次のセルの末尾の`| jq .`の部分をコメントアウトして実行して下さい。

In [ ]:
curl -sf http://localhost:8083/connectors/$kafka_connect_name | jq .

### タイムスタンプ

ブローカに送信されたセンサーデータの送信時刻をオブジェクトストレージに保存するKafka Connectを登録します。タイムスタンプの保存が不要な場合はこの節の実行をスキップできます。

In [ ]:
$script_dir/setup-sink-minio.sh -T -t $topic_name -b $bucket_name \
    -u $endpoint_url -A "$access_key" -S "$secret_key" \
    -n ${kafka_connect_name}-timestamp

登録状況を確認します。`jq`コマンドが利用できない場合は、次のセルの末尾の`| jq .`の部分をコメントアウトして実行して下さい。

In [ ]:
curl -sf http://localhost:8083/connectors/${kafka_connect_name}-timestamp | jq .